# LangChain: Agents

Agents can receive a query by making use of a set of tools or resources at its disposal. These tools can include access to web search, Wikipedia, complex Mathematical libraries, LLMs, etc.

`Wikipedia-API` is easy to use Python wrapper for Wikipedias’ API. It supports extracting texts, sections, links, categories, translations, etc from Wikipedia. Documentation provides code snippets for the most common use cases.

In [8]:
%pip install -U wikipedia


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Built-in LangChain tools ✅

In [9]:
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import load_tools, initialize_agent
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
# from langchain.agents.agent_toolkits import create_python_agent

from utils import SaladChatOllama,MODEL

In [10]:
llm = SaladChatOllama(temperature=0,model=MODEL)

In [57]:
# Used to answer math questions and wikipedia articles related contents.
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [58]:
agent_predict = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [59]:
print(agent_predict("What is the  15% of 100?"))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the  15% of 100?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What is the  15% of 100?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:SaladChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'type': 'string'}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an ac

In [60]:
question = "Dr.APJ Abdul Kalam has served as President of India. What was his tenure as President of India ?"
result = agent_predict(question)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Dr.APJ Abdul Kalam has served as President of India. What was his tenure as President of India ?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Dr.APJ Abdul Kalam has served as President of India. What was his tenure as President of India ?",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:SaladChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search que

## Create your Own Tool ✅

In [76]:
import random
from langchain.agents import tool

In [79]:
@tool
def get_inspiring_quote(text: str) -> str:
    """Returns a random inspirational quote from the below quotes."""
    quotes = [
        "Believe you can and you're halfway there. -Theodore Roosevelt",
        "The only limit to our realization of tomorrow will be our doubts of today. -Franklin D. Roosevelt",
        "Success is not final, failure is not fatal: It is the courage to continue that counts. -Winston Churchill",
        "The only way to do great work is to love what you do. -Steve Jobs",
        "In the middle of every difficulty lies opportunity. -Albert Einstein"
    ]

    return random.choice(quotes)

In [91]:
agent= initialize_agent(
    tools + [get_inspiring_quote],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

*To Note*:

The agent may occasionally arrive at an inaccurate conclusion (agents are continuously evolving and improving!).

If such a situation arises, please consider rerunning the query for a more accurate response.

In [92]:
try:
    result = agent("Give me a motivational quote for today")
except:
    print("exception on external access")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Give me a motivational quote for today"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Give me a motivational quote for today",
  "agent_scratchpad": "",
  "stop": [
    "Observation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:SaladChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nCalculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {'query': {'title': 'Query', 'type': 'string'}}\nget_inspiring_quote: get_inspiring_quote(text: str) -> s

In [93]:
print(result)

{'input': 'Give me a motivational quote for today', 'output': ' {\n  "action": "get_inspiring_quote",\n  "action_input": "A beautiful day begins with a beautiful mindset."\n}'}
